In [ ]:
import os
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

os.chdir("..")
os.makedirs("./cache", exist_ok=True)

from datasets import ImageNet_LT
from models import *
from trainer import load_clip_to_cpu
from utils.evaluator import PGD

from utils.config import _C as cfg

cfg.defrost()
cfg.merge_from_file('./configs/data/imagenet_lt.yaml')
cfg.merge_from_file('./configs/model/clip_vit_b16.yaml')
cfg.adaptformer = True

if not torch.cuda.is_available():
    device = torch.device("cpu")
elif cfg.gpu is None:
    device = torch.device("cuda")
else:
    torch.cuda.set_device(cfg.gpu)
    device = torch.device("cuda:{}".format(cfg.gpu))

cfg.model_dir = './output/imagenet_lt_clip_vit_b16_adaptformer_True'

In [ ]:
if cfg.backbone.startswith("CLIP"):
    mean = [0.48145466, 0.4578275, 0.40821073]
    std = [0.26862954, 0.26130258, 0.27577711]
else:
    mean = [0.5, 0.5, 0.5]
    std = [0.5, 0.5, 0.5]

transform_test = transforms.Compose([
    transforms.Resize(cfg.resolution * 8 // 7),
    transforms.CenterCrop(cfg.resolution),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_dataset = ImageNet_LT(root=cfg.root, train=True, transform=transform_test)
num_classes = train_dataset.num_classes
cls_num_list = train_dataset.cls_num_list
test_dataset = ImageNet_LT(root=cfg.root, train=False, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

clip_model = load_clip_to_cpu(cfg.backbone, cfg.prec)
model = PeftModelFromCLIP(cfg, clip_model, num_classes)
model.to(device)
tuner = model.tuner
head = model.head

load_path = os.path.join(cfg.model_dir, "checkpoint.pth.tar")
checkpoint = torch.load(load_path, map_location=device)
tuner_dict = checkpoint["tuner"]
head_dict = checkpoint["head"]
tuner.load_state_dict(tuner_dict)
head.load_state_dict(head_dict)

In [ ]:
def visualize_images(original_images, adversarial_images, labels, adv_labels, class_names):
    """
    Visualizes original and adversarial images side by side.
    """
    batch_size = original_images.size(0)
    for i in range(batch_size):
        original = original_images[i].permute(1, 2, 0).cpu().numpy()
        adversarial = adversarial_images[i].permute(1, 2, 0).cpu().numpy()
        
        fig, axes = plt.subplots(1, 2, figsize=(8, 4))
        axes[0].imshow((original * std + mean).clip(0, 1))
        axes[0].set_title(f"Original: {class_names[labels[i].item()]}")
        axes[0].axis("off")

        axes[1].imshow((adversarial * std + mean).clip(0, 1))
        axes[1].set_title(f"Adversarial: {class_names[adv_labels[i].item()]}")
        axes[1].axis("off")

        plt.tight_layout()
        plt.show()


In [ ]:
epsilon = 8 / 255  # 攻击强度
alpha = 2 / 255  # 每步扰动量
num_iterations = 2  # 攻击迭代次数

model.eval()  # 切换到评估模式
class_names = test_dataset.classnames  # 类别名称

for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    
    # 原始预测
    with torch.no_grad():
        original_logits = model(images)
        original_preds = torch.argmax(original_logits, dim=1)

    # 对抗样本生成
    adversarial_images = PGD(images, labels, model, eps=epsilon, alpha=alpha, steps=num_iterations)
    
    # 对抗样本预测
    with torch.no_grad():
        adversarial_logits = model(adversarial_images)
        adversarial_preds = torch.argmax(adversarial_logits, dim=1)

    # 可视化前10张图片
    visualize_images(
        original_images=images[:10],
        adversarial_images=adversarial_images[:10],
        labels=original_preds[:10],
        adv_labels=adversarial_preds[:10],
        class_names=class_names
    )
    
    break  # 仅显示一批数据


In [ ]:
def evaluate_model_under_attack(model, data_loader, epsilon, alpha, num_iter):
    """
    Evaluates model accuracy under PGD attack.
    """
    correct = 0
    total = 0

    for images, labels in tqdm(data_loader):
        images, labels = images.to(device), labels.to(device)

        # 生成对抗样本
        adversarial_images = PGD(model, images, labels, eps=epsilon, alpha=alpha, num_iter=num_iter)

        # 预测
        with torch.no_grad():
            outputs = model(adversarial_images)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Accuracy under PGD attack (ε={epsilon}): {accuracy:.2f}%")
    return accuracy



In [ ]:
# 评估模型在对抗样本上的性能
evaluate_model_under_attack(model, test_loader, epsilon, alpha, num_iterations)